In [5]:
import pandas as pd
import openpyxl

In [6]:
df3 = pd.read_csv('../données_brut/export_rdv_2023-01-01-2023-12-31.csv', sep=';')
df3['Age'] = 2024 - pd.to_datetime(df3['Date de naissance']).dt.year
new_df = pd.DataFrame({'Code_postal': df3['Code postal'], 'Statut': df3['Statut'], 
                       'Motif du RDV': df3['Motif du RDV'], 'Age': df3['Age'], 'Id': df3['Doctolib Patient ID']})
day_map = {
    'Absent excusé': 'Absent excusé',
    'Absent non excusé': 'Absent non excusé',
    'Vu' : 'Présent',
    'À déplacer': 'Absent excusé',
    'À venir': 'Absent non excusé',
    'En attente': 'Présent',
    'En consultation': 'Présent'    
}
new_df['Statut'] = new_df['Statut'].map(day_map)

/tmp/ipykernel_55534/2237332865.py:1: DtypeWarning: Columns (45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('../données_brut/export_rdv_2023-01-01-2023-12-31.csv', sep=';')


In [7]:
new_df.head()

,Code_postal,Statut,Motif du RDV,Age,Id
0,35740,Absent excusé,Adulte - Consultation de médecine générale,26.0,1733349697
1,75013,Présent,Première consultation de gynécologie,34.0,2518302311
2,92100,Présent,Consultation de suivi d'allergologie,29.0,2435238586
3,93300,Présent,Urgence,31.0,2518370353
4,75013,Absent excusé,Adulte - Consultation de médecine générale,34.0,2053339522


In [46]:
df = new_df

In [48]:
import pandas as pd


# Calcul du taux de Absent excusé
df['Absent_non_excuse'] = df['Statut'] == 'Absent non excusé'

# Calcul du taux
df['Psych'] = df['Motif du RDV'].str.contains("psychiatrie")

# GroupBy Code_postal et calcul des métriques
grouped = df.groupby('Code_postal').agg(
    taux_absent_non_excuse=('Absent_non_excuse', 'mean'),
    taux_Psych=('Psych', 'mean'),
    age_moyen=('Age', 'mean'),
    count=('Code_postal', 'size')
).reset_index()

grouped2 = df.groupby('Id').agg(
    Code_postal=('Code_postal', 'first'),
    taux_absent_non_excuse=('Absent_non_excuse', 'mean'),
    taux_Psych=('Psych', 'mean'),
    age_moyen=('Age', 'mean'),
).reset_index()

grouped22 = grouped2.groupby('Code_postal').agg(
    taux_absent_non_excuse=('taux_absent_non_excuse', 'mean'),
    taux_Psych=('taux_Psych', 'mean'),
    age_moyen=('age_moyen', 'mean'),
    count=('Code_postal', 'size')
).reset_index()

grouped = grouped[grouped['count'] > 3]
grouped22 = grouped22[grouped22['count'] > 3]


In [49]:
grouped = grouped[grouped['Code_postal'].astype(str).str.startswith(('75', '94', '93', '92'))].drop('taux_Psych', axis=1)
grouped22 = grouped22[grouped22['Code_postal'].astype(str).str.startswith(('75', '94', '93', '92'))].drop('taux_Psych', axis=1)

print(grouped)
print(grouped22)

     Code_postal  taux_absent_non_excuse  age_moyen  count
796        75001                0.075758  29.272727     66
797        75002                0.037736  32.320755     53
798        75003                0.068966  34.448276    116
799        75004                0.067548  34.459380    681
800        75005                0.090639  33.671664    673
...          ...                     ...        ...    ...
1324       94520                0.400000  35.400000      5
1325       94550                0.241379  32.379310     29
1326       94600                0.111111  33.320988     81
1327       94700                0.065574  39.344262    122
1328       94800                0.078049  34.497561    205

[139 rows x 4 columns]
     Code_postal  taux_absent_non_excuse  age_moyen  count
796        75001                0.052294  32.394737     38
797        75002                0.036585  32.073171     41
798        75003                0.067647  33.588235     85
799        75004                

In [50]:
grouped.to_csv("../donnees_nettroye/2023_code_post_gener_per_rdv.csv", index=False)
grouped22.to_csv("../donnees_nettroye/2023_code_post_gener_per_patient.csv", index=False)

In [51]:
grouped.head()

grouped22.head()

,Code_postal,taux_absent_non_excuse,age_moyen,count
796,75001,0.052294,32.394737,38
797,75002,0.036585,32.073171,41
798,75003,0.067647,33.588235,85
799,75004,0.068648,33.353808,408
800,75005,0.097449,33.696970,463


In [52]:
grouped.head()

,Code_postal,taux_absent_non_excuse,age_moyen,count
796,75001,0.075758,29.272727,66
797,75002,0.037736,32.320755,53
798,75003,0.068966,34.448276,116
799,75004,0.067548,34.459380,681
800,75005,0.090639,33.671664,673
